Logistic regression, using scikit-learn and statsmodels (based on introduction to statistical learning)

In [90]:
import numpy as np 
import pandas as pd
import statsmodels.api as sm 
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import seaborn as sns

In [91]:
#load datasets
large = pd.read_csv('../datasets/full_cleaned_dataset.csv')
small = pd.read_csv('../datasets/1std_dataset.csv')

In [92]:
#dropping the z_score column, so that small is a subset of large
small.drop(columns=['z_score'], inplace=True) 

#drop any non-numeric colums from both datasets
#getting lists of numeric columns
numeric_columns = large.select_dtypes(include=np.number).columns

#dropping non-numeric columns from large and small datasets
large = large[numeric_columns]
small = small[numeric_columns]

"""#turn dfs in numpy arrays
large = large.to_numpy()
small = small.to_numpy()"""

'#turn dfs in numpy arrays\nlarge = large.to_numpy()\nsmall = small.to_numpy()'

In [96]:
print(small['distressed'])

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
662    1.0
663    1.0
664    1.0
665    1.0
666    1.0
Name: distressed, Length: 667, dtype: float64


In [94]:
# logistic regression
scaler = StandardScaler()
X = scaler.fit_transform(small.copy())
y = small.distressed == 1
glm = sm.GLM(y, X, family=sm.families.Binomial())
results = glm.fit()

print(results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:             distressed   No. Observations:                  667
Model:                            GLM   Df Residuals:                      564
Model Family:                Binomial   Df Model:                          102
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -2.4333e-09
Date:                Fri, 04 Apr 2025   Deviance:                   4.8667e-09
Time:                        16:52:07   Pearson chi2:                 2.43e-09
No. Iterations:                    27   Pseudo R-squ. (CS):             0.1219
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.0004   1.56e+04  -2.56e-08      1.0

c:\Users\bicic\Documents\bachelor\.venv\Lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
c:\Users\bicic\Documents\bachelor\.venv\Lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
c:\Users\bicic\Documents\bachelor\.venv\Lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
c:\Users\bicic\Documents\bachelor\.venv\Lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may no

The results if this first model are bad. The low R2 and all P-values being 1 indicated that Logistic regerssion might not be a good fit for our data.

There is a perfect spearation warning. This could be solved by using a different model (Firth logistic regression is usually recommended, but it's not part of the libraries I'm using). Another option would be to remove variables that are causing the bias. 

In [95]:
print(mapper)

DataFrameMapper(drop_cols=[],
                features=[(Index(['cashAndCashEquivalents', 'shortTermInvestments',
       'cashAndShortTermInvestments', 'netReceivables',
       'inventory_balance_sheet', 'otherCurrentAssets', 'totalCurrentAssets',
       'propertyPlantEquipmentNet', 'goodwill', 'intangibleAssets',
       ...
       'incomeTaxExpense', 'netIncome_income_statement',
       'netIncomeRatio_income_statement', 'eps', 'epsdiluted',
       'weightedAverageShsOut', 'weightedAverageShsOutDil', 'distressed',
       'year', 'marketcap'],
      dtype='object', length=105),
                           StandardScaler())])
